# In this notebook
- Randomized iterative methods (Gower's paper)
- Using them to linear solve matrices AX = B
- ADI 

All are tested very loosely with small matrices. A more formal unit test suite can be found in the project in `../src`.

In [1]:
# usings and imports
using Random
using LinearAlgebra
using SparseArrays
using Elliptic
# using Plots


## Random iterative method (parameterized) 

Solves $Ax = b$

In [2]:
function rand_linsolve(A,b,B, Stype, seed=nothing,verbose=false)
    """
    Solves using randomized kaczmarz.
    Randomly selects a row per iteration to update x with
    
    INPUT:  matrix A mxn, 
            vector b mx1,
            matrix B nxn
            string Stype: "coordvec", ""
            
    OUTPUT: vector x nx1 that solves Ax = b
    """
    m,n = size(A)
    x0 = ones(n)
    sols = [x0] # list of x_0 ... x_n
    errs = zeros(0)
    if seed != nothing
        Random.seed!(seed)
    end
    @assert Stype == "coordvec"
    
    Binv = inv(B)
    xprev = x0
    while(true)
        
        i = rand(1:m)
        
        # S is a unit coordinate vector (temporary)
        S = zeros(m)
        S[i] = 1
        
        C = Binv * A' * S * pinv(S' * A * Binv * A' * S) * S'      
        res = A * xprev - b
        xnew = xprev - C * res
  
        push!(sols, xnew)
        err = norm(A * xnew - b)
        push!(errs,err)
        if err < 1e-5
            break
        end
        
        xprev = xnew
    end
    
    if verbose
        return sols,errs
    else
        return sols[end]
    end
end

rand_linsolve (generic function with 3 methods)

In [3]:
seed = 0

A = rand(Float64,30,50)
b = rand(Float64,30)
B = I # n x n 
# display("text/plain",A)
# display("text/plain",b)
# display("text/plain",B)

sols,errs = rand_linsolve(A,b, B,"coordvec", seed,true)
println("done")

done


In [4]:
println(size(errs))
# println(errs)
numiters= size(errs)[1]
plot(1:numiters, errs[:,1])

(10612,)


UndefVarError: UndefVarError: plot not defined

In [5]:
function rand_matsolve(A,B,seed=nothing)
    """
    Helper function that stacks together multiple Ax=b solvers
    INPUT: 
            A: matrix mxn
            B: matrix mxk
    OUTPUT: X: matrix such that AX = B
    """
    
    m1,n = size(A)
    m2,k = size(B)
    sol = zeros(n,k)
    @assert m1==m2 
    
    s(b) = rand_linsolve(A,b,I,"coordvec", seed)
    return mapslices(s,B,dims=[1])
    
end

rand_matsolve (generic function with 2 methods)

In [10]:
A = rand(Float64,30,50)
B = rand(Float64,30,60)
Bparam = I # n x n
m1,n = size(A)
m2,k = size(B)
sol = zeros(n,k)

X = rand_matsolve(A,B)
println("done")
println(norm(A * X - B))

done
7.679277571487114e-5


In [7]:

function adi_parameters(a,b,c,d,J)
    p = zeros(J)
    q = zeros(J)
    gamma = abs((c-a) * (d-b)  / (c-b)*(d-a))
    alpha = -1 + 2*gamma + 2 * sqrt(gamma^2 - gamma)
    kappa = sqrt(1 - 1 / (alpha^2))

    TT(t) = (-alpha * t -1)/(t + alpha)

    for j = 1:J
        z = (2 * j + 1) / (2 * J) * Elliptic.K(kappa)
        t = -alpha * Elliptic.Jacobi.dn(z,kappa)
        p[j] = TT(t)
        q[j] = TT(-t)
    end

    return p,q
end

adi_parameters (generic function with 1 method)

## ADI

Solves $AX - XB = N$ for low rank $N$

In [63]:
function adi(A,B,F,N,p,q)
    m,m2 = size(A)
    n,n2 = size(B)
    
    @assert m==m2
    @assert n==n2
    
    sols = []
    Xprev = zeros((m,n)) 
    for i = 1:N
        Ahalf = (A - p[i] * I)
        Bhalf = Xprev * (B - p[i] * I) + F
#         Xhalf = rand_matsolve(Ahalf,Bhalf) 
        Xhalf = Ahalf \ Bhalf

        Asolve = (B - q[i] * I)
        Bsolve = (A - q[i] * I) * Xhalf - F
#         X = (rand_matsolve(Asolve', Bsolve'))'
        X = (Asolve' \ Bsolve')'
            
        Xprev = X 
        push!(sols,X)
    end    
    return sols
end

adi (generic function with 1 method)

In [64]:
#--- Small Test
temp = rand(5,5)
Q,R = qr(temp)
Aevals = [1., 2., 3., 4., 5.]
Bevals = [11., 12., 13., 14., 15.]
A = Q * Diagonal(Aevals) * Q'
B = Q * Diagonal(Bevals) * Q'
F = rand(Float64,5,5)

a = minimum(Aevals)
b = maximum(Aevals)
c = minimum(Bevals)
d = maximum(Bevals)

15.0

In [65]:
#--- Run Small Test
N = 
p,q = adi_parameters(a,b,c,d,N)
sols = adi(A,B,F,N,p,q)
X = sols[end]
print(norm(A * X - X * B - F))
#---

MethodError: MethodError: no method matching (::Colon)(::Int64, ::Tuple{Array{Float64,1},Array{Float64,1}})
Closest candidates are:
  Colon(::T<:Real, ::Any, !Matched::T<:Real) where T<:Real at range.jl:40
  Colon(::A<:Real, ::Any, !Matched::C<:Real) where {A<:Real, C<:Real} at range.jl:10
  Colon(::T, ::Any, !Matched::T) where T at range.jl:39
  ...

In [39]:
@time adi(A,B,F,N,p,q)

  0.001098 seconds (103 allocations: 19.313 KiB)


1